# 2022 Flatiron Machine Learning x Science Summer School

## Step 9: Implement symbolic discriminator

In this step, we train a symbolic discriminator to regularize the latent features of our `SRNet`.

The aim is that the latent features are incentivized to resemble functions that are easily discoverable with symbolic regression.

The plan is training a Wasserstein Generative Adversarial Network (WGAN) with gradient penalty. The advantage of a WGAN over a regular GAN is that it avoids mode collapse and vanishing gradients. The WGAN requires the discriminator to be 1-Lipschitz continuous, which is softly enforced with gradient penalty (check https://www.youtube.com/watch?v=v6y5qQ0pcg4).

First, we are going to apply a WGAN to the simplest possible formulation of our problem:

* Two input features

* Bottleneck

* Fixed $\alpha$ mask values

* Pretrained weights

* Only target functions as real data

Notes:

* We probably do not want to shuffle the input data

Questions:

* Do we feed the discriminator only latent feature values or also input feature values?

* Do we want different batches, i.e. the `SRNet` and real functions evaluated at different positions?

**TODO**:

* Try `LeakyReLU`, `BatchNorm`

* Check https://github.com/aladdinpersson/Machine-Learning-Collection

* Check `prior.jl`

* Check Miles' code (https://colab.research.google.com/drive/1IoJh46JV8EaxWjF3kDS51MZyAdXKzBxc?usp=sharing#scrollTo=4G3ZNAUeCxMX)

* Check preevaluation vs. online evaluation

* Check batch_size

* Consider input features?

* Which epoch to start discriminator?

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

import torch
import wandb

from srnet import SRNet, SRData
import srnet_utils as ut

In [2]:
# plot losses
save_names = ["srnet_model_F00_bn_norm_sd_1e-04_test.pkl"]
save_path = "models"
models = ut.plot_losses(save_names, save_path="models", excl_names=["max"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
# load data
data_path = "data_1k"

in_var = "X00"
lat_var = "G00"
target_var = "F00"

mask_ext = ".mask"
masks = joblib.load(os.path.join(data_path, in_var + mask_ext))

train_data = SRData(data_path, in_var, lat_var, target_var, masks["train"])
val_data = SRData(data_path, in_var, lat_var, target_var, masks["val"])

In [7]:
model_name = "srnet_model_F00_bn_norm_sd_1e-04_test"
model_path = "models"
model_ext = ".pkl"

model = ut.load_model(model_name + model_ext, model_path, SRNet)

with torch.no_grad():
    preds, acts = model(train_data.in_data, get_lat=True)
    
all_nodes = ut.get_node_order(acts, show=True)

print(model.layers1.alpha.detach().cpu().numpy()[all_nodes])
print(model.layers1.norm(model.layers1.alpha).detach().cpu().numpy()[all_nodes])

print("")

[1.8365434, 0.94673306, 0.1891221]
[2, 1, 0]
[[-1.4036835  -1.0781626 ]
 [-1.9185202   0.82522213]
 [ 1.3652941  -0.46780935]]
[[0.58066916 0.41933084]
 [0.7490023  0.25099772]
 [0.7104324  0.28956768]]



In [8]:
nodes = all_nodes[:3]

In [9]:
# select plotting data
x_data = train_data.in_data[:,0]
y_data = train_data.in_data[:,1]
z_data = [
    ("target", train_data.target_data),
    #("x**2", x_data**2), 
    #("cos(y)", np.cos(y_data)), 
    #("x*y", x_data * y_data),
]
plot_size = train_data.target_data.shape[0]

In [10]:
ut.plot_acts(x_data, y_data, z_data, acts=acts, nodes=nodes, model=model, bias=True, nonzero=False, agg=False, plot_size=plot_size)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
corr_data = [
    ("x**2", x_data**2), 
    ("cos(y)", np.cos(y_data)), 
    ("x*y", x_data * y_data),
]

In [19]:
ut.node_correlations(acts, nodes, corr_data)


Node 2
corr(n2, x**2): 0.9999
corr(n2, cos(y)): -0.0170
corr(n2, x*y): 0.1786

Node 1
corr(n1, x**2): 0.1706
corr(n1, cos(y)): -0.0622
corr(n1, x*y): 0.9999

Node 0
corr(n0, x**2): -0.0153
corr(n0, cos(y)): 0.9997
corr(n0, x*y): -0.0672


In [13]:
fig, ax = plt.subplots()

ax.scatter(x_data, x_data**2)
ax.scatter(x_data, acts[:,2])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# select plotting data
x_data = train_data.in_data[:,0]
y_data = train_data.in_data[:,1]
z_data = [
    #("target", train_data.target_data),
    #("x**2", x_data**2), 
    #("cos(y)", np.cos(y_data)), 
    ("x*y", x_data * y_data),
]
plot_size = train_data.target_data.shape[0]

In [15]:
ut.plot_acts(x_data, y_data, z_data, acts=acts, nodes=[1], model=model, bias=True, nonzero=False, agg=False, plot_size=plot_size)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig, ax = plt.subplots()

ax.scatter(y_data, np.cos(y_data))
ax.scatter(y_data, acts[:,0])

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …